In [ ]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install ortools

In [ ]:
#importing dataset
import pandas as pd
import datetime as dt
rainday_clean = '/content/drive/MyDrive/Infrastructure Systems Optimization Group/Taxi Data and Code/Rainday/Clean/cleaned_rainday.csv'
df1 = pd.read_csv(rainday_clean)


In [ ]:
#removing columns not used in optimization
df = df1.loc[:,['lpep_pickup_datetime','Pickup_latitude','Pickup_longitude','Dropoff_latitude','Dropoff_longitude','Trip_distance','Passenger_count']]

#constraining pickup locations to upper Manhattan
lat_upper = 40.882279
lat_lower = 40.790053
long_upper = -73.931275
long_lower = -73.985336

#constraining to 1-hour time range
df['lpep_pickup_datetime'] = pd.to_datetime(df['lpep_pickup_datetime'])
start_time = pd.Timestamp('01/10/2016 12:00:01 AM')
end_time = pd.Timestamp('01/10/2016 01:00:01 AM')

#only 1 passenger in shareable rides
passenger_max = 1

df = df[(df['Pickup_latitude'] >= lat_lower) & (df['Pickup_latitude'] <= lat_upper) & (df['Pickup_longitude'] >= long_lower) & (df['Pickup_longitude'] <= long_upper)]
df = df[(df['lpep_pickup_datetime'] >= start_time) & (df['lpep_pickup_datetime'] <= end_time) & (df['Passenger_count'] <= passenger_max)]
df = df.reset_index(drop=True)

print(df.head())
print('rows:', len(df))

  lpep_pickup_datetime  Pickup_latitude  Pickup_longitude  Dropoff_latitude  \
0  2016-01-10 00:00:13        40.811588        -73.954407         40.805779   
1  2016-01-10 00:00:24        40.809189        -73.944344         40.820637   
2  2016-01-10 00:00:37        40.804153        -73.936951         40.813023   
3  2016-01-10 00:00:50        40.797146        -73.949242         40.711472   
4  2016-01-10 00:00:50        40.820671        -73.954758         40.806561   

   Dropoff_longitude  Trip_distance  Passenger_count  
0         -73.965462           0.83                1  
1         -73.936188           0.90                1  
2         -73.900864           2.38                1  
3         -73.945480          13.43                1  
4         -73.965065           1.10                1  
rows: 482


In [ ]:
#specify the number of trips we want to analyze (trips is chronologically ordered)
# num_trip = len(df)
num_trip = 100

In [ ]:
from ortools.linear_solver import pywraplp
import math
import numpy as np

In [ ]:
Pickup_latitude = df['Pickup_latitude']
Pickup_longitude = df['Pickup_longitude']
Dropoff_latitude = df['Dropoff_latitude']
Dropoff_longitude = df['Dropoff_longitude']
Trip_distance = df['Trip_distance']
Pickup_time = df['lpep_pickup_datetime']

In [ ]:
# define the euclidean_distance function (with conversion to miles)
mi_per_deg_lat = 69.4 #miles per degree of latitude
mi_per_deg_long = 52.356 #miles per degree of longitude at a central point in the NYC region (approx)

def euclidean_distance(lat1, lon1, lat2, lon2):
    # Pythagorean formula
    return math.sqrt((mi_per_deg_lat*(lat1 - lat2))**2 + (mi_per_deg_long*(lon1 - lon2))**2)

# initialize an empty matrix for pickup distances
Pickup_matrix = np.zeros((num_trip, num_trip))

# calculate distances for the entire dataset
for i in range(num_trip):
    for j in range(num_trip):
        distance = euclidean_distance(Pickup_latitude[i], Pickup_longitude[i],Pickup_latitude[j], Pickup_longitude[j])
        Pickup_matrix[i, j] = distance

# Now, Pickup_matrix contains the distances between the origin points of every possible pair of trips
# print(Pickup_matrix)

In [ ]:
# initialize an empty matrix for dropoff distances
Dropoff_matrix = np.zeros((num_trip, num_trip))

# calculate distances for the entire dataset, considering the "trip distance" constraint
for i in range(num_trip):
    for j in range(num_trip):
        distance = euclidean_distance(Dropoff_latitude[i], Dropoff_longitude[i],Dropoff_latitude[j], Dropoff_longitude[j])
        # add to matrix as i,j element
        Dropoff_matrix[i, j] = distance

# Dropoff_matrix contains the distances between the end points of every possible pair of trips
# print(Dropoff_matrix)

In [ ]:
Timing_matrix = np.zeros((num_trip, num_trip))

time_constraint = 60
# calculate start time difference between every trip
for i in range(num_trip):
    for j in range(num_trip):
        time_difference = Pickup_time[j] - Pickup_time[i]
        #convert to minutes
        minute_difference = time_difference.total_seconds()/60
        #add to matrix
        Timing_matrix[i, j] = minute_difference

# Now, Timing_matrix contains the time gaps for every pair of trips
# print(Timing_matrix)

In [ ]:
solver=pywraplp.Solver('SolverAssignmentProblemMIP',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)


In [ ]:
# decision variables x_ij are initiated as booleans -
# they will take a value of 1 when the trips i and j are paired and a value of 0 when i and j are not paired
x={}
for i in range(num_trip):
  for j in range(num_trip):
    if i==j:
      x[i,j]=0
    elif i!=j:
      x[i,j]=solver.BoolVar('x[%i,%i]' % (i,j))


In [ ]:
#objective function is the sum of all elements in the x_ij table
# so it will be two times the number of pairs (since x_ij and x_ji will both be 1)
solver.Maximize(solver.Sum([x[i, j] for i in range(num_trip) for j in range(num_trip)]))

In [ ]:
#constraints
Pickup_Dmax=0.25 # to match, trip must originate 0.25 miles apart
Dropoff_Dmax=0.5  # and their destinations must be 0.5 miles apart (assuming one passenger can be dropped off ~5min before the other)
tmax = 20 #their trips must begin within 20 min of each other
for i in range(num_trip):
  for j in range(num_trip):
    solver.Add(x[i,j]*float(Pickup_matrix[i][j]) <= Pickup_Dmax)
    solver.Add(x[i,j]*float(Dropoff_matrix[i][j]) <= Dropoff_Dmax)
    solver.Add(x[i,j]==x[j,i])
    solver.Add(x[i,j]*float(Timing_matrix[i][j]) <= tmax)

In [ ]:
# to make sure that each trip is matched at most with one other trip
# we constrain each row to only have one entry of 1
for i in range(num_trip):
    solver.Add(solver.Sum([x[i, j] for j in range(num_trip)]) <= 1)
    solver.Add(solver.Sum([x[i, j] for i in range(num_trip)]) <= 1)

In [ ]:
# solver.constraints()
sol = solver.Solve()

In [ ]:
objective_value = solver.Objective().Value()
print('Maximum pairs =', objective_value / 2)

percent_matched = objective_value / num_trip
print('Percent of trips matched in time range: ', percent_matched*100, '%')

Maximum pairs = 16.0
Percent of trips matched in time range:  32.0 %


In [ ]:
# print out the matches and their original info for easier validation
for i in range(num_trip):
    for j in range(num_trip):
        if i != j and x[i, j].solution_value() > 0:
            print('Trip %d is matched with Trip %d, Distance between origins = %f mi' % (i + 1, j + 1, Pickup_matrix[i][j]))
            print('Trip %d originates at %.6f N, and %.6f W at %s and ends at %.6f N, and %.6f W (distance: %.3f)' % (i+1,df.iloc[i,1],df.iloc[i,2],df.iloc[i,0],df.iloc[i,3],df.iloc[i,4],df.iloc[i,5]))
            print('Trip %d originates at %.6f N, and %.6f W at %s and ends at %.6f N, and %.6f W (distance: %.3f) \n' % (j+1,df.iloc[j,1],df.iloc[j,2],df.iloc[j,0],df.iloc[j,3],df.iloc[j,4],df.iloc[j,5]))

Trip 2 is matched with Trip 26, Distance between origins = 0.202332 mi
Trip 2 originates at 40.809189 N, and -73.944344 W at 2016-01-10 00:00:24 and ends at 40.820637 N, and -73.936188 W (distance: 0.900)
Trip 26 originates at 40.806690 N, and -73.942352 W at 2016-01-10 00:03:14 and ends at 40.815926 N, and -73.941025 W (distance: 0.850) 

Trip 7 is matched with Trip 12, Distance between origins = 0.076008 mi
Trip 7 originates at 40.802292 N, and -73.949394 W at 2016-01-10 00:01:02 and ends at 40.801018 N, and -73.937737 W (distance: 0.790)
Trip 12 originates at 40.803234 N, and -73.948654 W at 2016-01-10 00:01:52 and ends at 40.805470 N, and -73.942261 W (distance: 0.600) 

Trip 8 is matched with Trip 59, Distance between origins = 0.241628 mi
Trip 8 originates at 40.817703 N, and -73.941818 W at 2016-01-10 00:01:12 and ends at 40.845276 N, and -73.934860 W (distance: 2.490)
Trip 59 originates at 40.814297 N, and -73.940865 W at 2016-01-10 00:06:58 and ends at 40.841507 N, and -73.940